In [1]:
import pandas as pd
import numpy as np
from difflib import get_close_matches

In [2]:
!pip install cvxpy

     |████████████████████████████████| 993kB 1.6MB/s eta 0:00:01     |████████████                    | 368kB 1.6MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.6MB 3.4MB/s eta 0:00:01     |██████████████████████          | 2.4MB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 153kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 215kB 4.5MB/s eta 0:00:01
  Created wheel for cvxpy: filename=cvxpy-1.1.1-cp36-cp36m-linux_x86_64.whl size=2498588 sha256=c1fe4a6dc2759ca4fe78f5be3ccf8998207fdf9599a334bb1454cfb34461c05d
  Stored in directory: /home/nbuser/.cache/pip/wheels/06/db/59/b5af93d86703e0903b9b94ccc300ac70daf9d273f13e6c0350
Successfully built cvxpy
  Created wheel for scs: filename=scs-2.1.2-cp36-cp36m-linux_x86_64.whl size=376094 sha256=b6ff74ce7aabb40d81ba8319a05fd77c5d3ebd725d44278f3f97feb13b02281c
  Stored in directory: /

In [3]:
import cvxpy as cp

In [4]:
#First, need a function that picks elements from list L with probabilies probs
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

# Examine wave split

In [5]:
df1=pd.read_csv('Data/outlaw_results_20200512.csv')[['Player','FPTS']]
df1 = df1.dropna()
df1.iloc[np.r_[0:2,-2:0]]

,Player,FPTS
0,Joel Dahmen,50.15
1,Nate Lashley,67.00
160,Quinn Carbol,7.20
161,Marcus Bantel,5.10


In [6]:
df2=pd.read_csv('Data/outlaw_results_20200513.csv')[['Player','FPTS']]
df2 = df2.dropna()
df2.iloc[np.r_[0:2,-2:0]]

,Player,FPTS
0,Joel Dahmen,57.70
1,Nate Lashley,31.05
157,Tyler Jessop,21.35
158,Marcus Bantel,-5.95


In [7]:
#Combine and separate R1 vs R2
df1.rename(columns={'FPTS':'FPTS_R1'},inplace=True)
df2.rename(columns={'FPTS':'FPTS_R2'},inplace=True)
df_waves = pd.merge(df1,df2,how='inner')
df_waves.rename(columns={'Player':'Name'},inplace=True)
df_waves.head()

,Name,FPTS_R1,FPTS_R2
0,Joel Dahmen,50.15,57.70
1,Nate Lashley,67.00,31.05
2,Brady Calkins,53.25,38.35
3,Taylor Montgomery,45.70,44.95
4,Colton Yates,36.75,41.65


In [8]:
#And tee times
#These are R2 tee times for 20200513
df_tees = pd.read_csv('Data/golf_tee_times.csv')
df_tees.head(3)

,Player,Tee Time,Hole,Tee,Other Players
0,"Adams, Cole",1:30 PM,1,Black,"Burdick, Conner + Vivolo, Luke"
1,"Alder, Preston",9:00 AM,1,Black,"Gulugian, Ari + Park, Joey"
2,"Alker, Steven",12:45 PM,10,Black,"Allan, Steve + Bobbitt, Redford"


In [9]:
#Create Name
df_tees['Name'] = df_tees.Player.str.split(', ',expand=True)[1] + " " + df_tees.Player.str.split(', ',expand=True)[0]

#And add AM vs PM
df_tees['Wave'] = np.where(df_tees['Tee Time'].str.contains('AM'),'AM','PM')
df_tees.head()

,Player,Tee Time,Hole,Tee,Other Players,Name,Wave
0,"Adams, Cole",1:30 PM,1,Black,"Burdick, Conner + Vivolo, Luke",Cole Adams,PM
1,"Alder, Preston",9:00 AM,1,Black,"Gulugian, Ari + Park, Joey",Preston Alder,AM
2,"Alker, Steven",12:45 PM,10,Black,"Allan, Steve + Bobbitt, Redford",Steven Alker,PM
3,"Allan, Steve",12:45 PM,10,Black,"Alker, Steven + Bobbitt, Redford",Steve Allan,PM
4,"Allred, Zach",8:42 AM,1,Black,"Fryer, Joe + Kuest, Peter",Zach Allred,AM


In [10]:
#Replace Player with Name from df1
from difflib import get_close_matches
#df['Name'] = ""
for i in np.arange(len(df_tees)):
    curr_name = df_tees['Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df_waves['Name'],cutoff=.8)[0]
        if new_name != curr_name:
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df_tees['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df_tees.head(5)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


NO MATCH: curr_name = Quinn Carbol, new_name = No Name
NO MATCH: curr_name = Lamar Carlile, new_name = No Name
Mitch Carlson replaced with Mitchel Carlson
Terry Cook (AM) replaced with Terry Cook
Samuel Cyr replaced with Sam Cyr
NO MATCH: curr_name = Patrick Fishburn, new_name = No Name
Jared Hack replaced with Jhared Hack
NO MATCH: curr_name = Rodney Hamblin, new_name = No Name
NO MATCH: curr_name = Scott Harrington, new_name = No Name
Matthew Liringis replaced with Matt Liringis
NO MATCH: curr_name = Manny Manzone, new_name = No Name
Doug Quinones replaced with Douglas Quinones
Oliver Roberts replaced with Ollie Roberts
NO MATCH: curr_name = Ben Shur, new_name = No Name
NO MATCH: curr_name = Nicholas Timm, new_name = No Name


,Player,Tee Time,Hole,Tee,Other Players,Name,Wave
0,"Adams, Cole",1:30 PM,1,Black,"Burdick, Conner + Vivolo, Luke",Cole Adams,PM
1,"Alder, Preston",9:00 AM,1,Black,"Gulugian, Ari + Park, Joey",Preston Alder,AM
2,"Alker, Steven",12:45 PM,10,Black,"Allan, Steve + Bobbitt, Redford",Steven Alker,PM
3,"Allan, Steve",12:45 PM,10,Black,"Alker, Steven + Bobbitt, Redford",Steve Allan,PM
4,"Allred, Zach",8:42 AM,1,Black,"Fryer, Joe + Kuest, Peter",Zach Allred,AM


In [11]:
df_waves = pd.merge(df_waves,df_tees[['Name','Wave']])
df_waves.head(3)

,Name,FPTS_R1,FPTS_R2,Wave
0,Joel Dahmen,50.15,57.70,PM
1,Nate Lashley,67.00,31.05,PM
2,Brady Calkins,53.25,38.35,PM


In [12]:
df_waves.rename(columns={'Wave':'Wave_R2'},inplace=True)
df_waves['Wave_R1'] = df_waves.Wave_R2.map({'PM':'AM','AM':'PM'})

df_waves.head(15)

,Name,FPTS_R1,FPTS_R2,Wave_R2,Wave_R1
0,Joel Dahmen,50.15,57.70,PM,AM
1,Nate Lashley,67.00,31.05,PM,AM
2,Brady Calkins,53.25,38.35,PM,AM
3,Taylor Montgomery,45.70,44.95,PM,AM
4,Colton Yates,36.75,41.65,PM,AM
5,Patrick Flavin,44.95,42.40,PM,AM
6,Nick Mason,36.45,40.70,PM,AM
7,Matt Picanso,38.60,24.65,PM,AM
8,JJ Spaun,36.45,40.35,PM,AM
9,KK Limbhasut,44.75,45.90,PM,AM


In [13]:
df_R1 = df_waves.groupby('Wave_R1')['FPTS_R1'].mean()
df_R1

Wave_R1
AM    31.098026
PM    19.862179
Name: FPTS_R1, dtype: float64

In [14]:
df_R2 = df_waves.groupby('Wave_R2')['FPTS_R2'].mean()
df_R2

Wave_R2
AM    27.491026
PM    32.247368
Name: FPTS_R2, dtype: float64

In [15]:
#find advantage of AM
am = (df_R1.loc['AM']+df_R2.loc['AM'])/2
pm = (df_R1.loc['PM']+df_R2.loc['PM'])/2
print(am,pm,am-pm)
 

29.294525978407563 26.054773954116065 3.239752024291498


In [16]:
ap = np.mean([df_R1.loc['AM'],df_R2.loc['PM']])
pa = np.mean([df_R1.loc['PM'],df_R2.loc['AM']])
print(ap,pa)

31.672697368421062 23.676602564102566


In [17]:
ap+pa

55.34929993252363

# Read in Data

In [18]:
#Read in DK Salaries
days = ['20200415','20200421','20200422','20200428','20200429','20200430',
        '20200504','20200505','20200506','20200512','20200513','20200514',
       '20200519','20200520','20200521','20200526','20200601']
df_sal_list = []
for d in days:
    df_sal_list.append(pd.read_csv('Data/DKSalaries_outlaw_{}.csv'.format(d)))
df_sal_list[0].head()

,Unnamed: 0,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,0,G,Jimmy Gunn (14482079) (LOCKED),Jimmy Gunn,14482079,G,10800,Arrowhead Classic,Golf,0.00
1,1,G,Alex Cejka (14482080) (LOCKED),Alex Cejka,14482080,G,10500,Arrowhead Classic,Golf,51.57
2,2,G,Carson Roberts (14482081) (LOCKED),Carson Roberts,14482081,G,10300,Arrowhead Classic,Golf,0.00
3,3,G,Nick Mason (14482082) (LOCKED),Nick Mason,14482082,G,10000,Arrowhead Classic,Golf,0.00
4,4,G,Andrew Yun (14482083) (LOCKED),Andrew Yun,14482083,G,9800,Arrowhead Classic,Golf,0.00


In [19]:
#Read in odds
days_odds = ['20200414','20200421','20200421','20200428','20200428','20200428',
             '20200504','20200504','20200504','20200512','20200512','20200512',
            '20200519','20200519','20200519','20200526','20200601']
df_odds_list = []
for d in days_odds:
    df_odds_list.append(pd.read_csv('Data/Outlaw_odds_{}.csv'.format(d)))
df_odds_list[-1].head()

,Name,Odds
0,Brady Calkins,7.0
1,Sahith Theegala,7.5
2,Tyler Weworski,9.5
3,Nicolo Galletti,11.0
4,Alistair Docherty,12.0


In [20]:
#Read in results
df_results_list = []
for d in days:
    df_results_list.append(pd.read_csv('Data/outlaw_results_{}.csv'.format(d)))
    
df_results_list[-1].head()

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,Unnamed: 6,Player,Roster Position,%Drafted,FPTS
0,1,1966096713,forfamily (2/2),0,381.35,G Sahith Theegala G Andrew Funk G Trevor Lamps...,NaN,Andrew Funk,G,34.64%,75.25
1,2,1965906979,CecPetes (11/30),0,354.65,G Alistair Docherty G Nicolo Galletti G Andrew...,NaN,Noah Hofman,G,33.17%,28.90
2,3,1962734147,benchico (2/6),0,349.80,G Alistair Docherty G Andrew Funk G Trevor Lam...,NaN,Trevor Lampson,G,28.80%,57.50
3,3,1965736764,Bentley888Bentley (34/150),0,349.80,G Alistair Docherty G Andrew Funk G Trevor Lam...,NaN,Brandon Bauman,G,26.61%,42.60
4,5,1965453594,ajkdrfc (4/78),0,348.85,G Alistair Docherty G Neil Johnson G Andrew Fu...,NaN,Daniel Hudson,G,24.99%,57.50


### Combine data
For each day, get one single df with Name, Salary, Odds, Win, FPTS

Note: Odds could be 5 or 500 for 5 to 1.  Check if all odds are over 100, and if so divide by 100

In [21]:
#Scratch work
i = len(df_odds_list)-1

df = df_sal_list[i][['Name','Salary']]
df.head()

#Get everyone's final FPTS
df_results = df_results_list[i]
df_results.rename(columns={'Player':'Name'},inplace=True)
df_results.head()

#Add FPTS
df = pd.merge(df,df_results[['Name','FPTS']],on='Name',how='left')
df.head()

#Only keep entries with FPTS > 0
df = df.dropna()
df = df[df.FPTS>0]
df.tail()

#Add in odds
df_odds = df_odds_list[i]
df_odds.head()
#Make sure odds are floats with no commas
df_odds.Odds = df_odds.Odds.astype(str).str.replace(',','').astype(float)
#Make sure odds are number:1, not 100*number:1
if df_odds.Odds.min()>100:
    df_odds.Odds = df_odds.Odds / 100

#Apply rstrip to Name since it might have a goofy extra white space
df_odds.Name = df_odds.Name.str.rstrip()

#Replace Name in df_odds with closest name in df_sal
for j in np.arange(len(df_odds)):
    curr_name = df_odds['Name'].iloc[j]
    try:
        new_name = get_close_matches(curr_name,df['Name'],cutoff=.8)[0]
        #if new_name != curr_name:
            #print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        #print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df_odds['Name'].iloc[j] = new_name

df_odds.head()

#Add odds to df_sal
df = pd.merge(df,df_odds,how='left')
df = df.dropna()
df.iloc[np.r_[0:8,-2:0]]

,Name,Salary,FPTS,Odds
0,Brady Calkins,10800,50.05,7.0
1,Tyler Weworski,10600,57.50,9.5
2,Sahith Theegala,10400,76.25,7.5
3,Alistair Docherty,10200,67.95,12.0
4,Mark Baldwin,10000,41.65,12.0
5,Nicolo Galletti,9800,55.15,11.0
6,Steven Alker,9700,45.90,17.0
7,Neil Johnson,9400,53.45,15.0
46,Anthony Quezada,6100,34.15,401.0
47,Adam Petrocelli,6000,1.60,751.0


In [22]:
#If someone has two separate odds, only keep lower ones
df.groupby(['Name','Salary','FPTS'])['Odds'].min().reset_index().sort_values(by='Salary',ascending=False).head(8)

,Name,Salary,FPTS,Odds
9,Brady Calkins,10800,50.05,7.0
44,Tyler Weworski,10600,57.50,9.5
35,Sahith Theegala,10400,76.25,7.5
3,Alistair Docherty,10200,67.95,12.0
20,Mark Baldwin,10000,41.65,12.0
29,Nicolo Galletti,9800,55.15,11.0
39,Steven Alker,9700,45.90,17.0
28,Neil Johnson,9400,53.45,15.0


In [ ]:
df_list = []
for i in range(len(df_sal_list)):
    df = df_sal_list[i][['Name','Salary']]

    #Get everyone's final FPTS
    df_results = df_results_list[i]
    df_results.rename(columns={'Player':'Name'},inplace=True)

    #Add FPTS
    df = pd.merge(df,df_results[['Name','FPTS']],on='Name',how='left')

    #Only keep entries with FPTS > 0
    df = df.dropna()
    df = df[df.FPTS>0]

    #Add in odds
    df_odds = df_odds_list[i][['Name','Odds']]
    #Make sure odds are floats with no commas
    df_odds.Odds = df_odds.Odds.astype(str).str.replace(',','').astype(float)
    #Make sur eodds are number:1, not 100*number:1
    if df_odds.Odds.min()>100:
        df_odds.Odds = df_odds.Odds / 100
    #Apply rstrip to Name since it might have a goofy extra white space
    df_odds.Name = df_odds.Name.str.rstrip()

    #Replace Name in df_odds with closest name in df_sal
    for j in np.arange(len(df_odds)):
        curr_name = df_odds['Name'].iloc[j]
        try:
            new_name = get_close_matches(curr_name,df['Name'],cutoff=.8)[0]
            #if new_name != curr_name:
                #print("{} replaced with {}".format(curr_name,new_name))
        except:
            new_name = 'No Name'
            #print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
        df_odds['Name'].iloc[j] = new_name

    #Add odds to df_sal
    df = pd.merge(df,df_odds,how='left')
    df = df.dropna()
    
    #Make sure name isn't repeated with 2 different odds
    df = df.groupby(['Name','Salary','FPTS'])['Odds'].min().reset_index().sort_values(by='Salary',ascending=False)
    #Add Win probability as well
    df['Win'] = 1 / (df.Odds+1)
    
    df_list.append(df)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
df_list[-1].iloc[np.r_[0:8,-4:0]]

# Now Optimize in different ways

In [ ]:
num_teams_deep = 3

In [ ]:
pd.DataFrame(data={day:np.zeros(num_teams_deep) for day in days})

In [ ]:
#start to record truly optimal teams
df_outlaw_optimals = pd.DataFrame(columns=['Team','Salary','FPTS'])
df_outlaw_optimals

In [ ]:
#Explore truly optimal teams
num_teams_deep = 1
df_scores = pd.DataFrame(data={day:np.zeros(num_teams_deep) for day in days})
#start to record truly optimal teams
df_outlaw_optimals = pd.DataFrame(columns=['Optimal Team','Salary','FPTS'])
opt_scores = []
for day, df in zip(days,df_list):
    print(day)
    df_now = df.copy()
    min_sal = 60
    max_sal = 12000
    df_now = df_now[(df_now.Salary>=min_sal)&(df_now.Salary<=max_sal)]
    opt_teams = []
    for i in range(num_teams_deep):
        n = len(df_now)
        x = cp.Variable(n,boolean=True)
        s = df_now.Salary.values
        w = df_now.Win.values
        o = df_now.Odds.values
        p = df_now.FPTS.values

        cap = 50000
        constraints = [cp.sum(x) == 6,
                     x*s <= cap]

        #Objective
        #obj = cp.Minimize(x*o)
        obj = cp.Maximize(x*p)

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve()  
        opt_team = df_now['Name'][x.value>.5].tolist()
        sal=df_now['Salary'][x.value>.5].sum()
        opt_teams = opt_teams + opt_team
        score = df_now[x.value>.5]['FPTS'].sum()
        print('  Team {} ({:.1f} pts): {}'.format(i+1,score,opt_team))
        df_scores.at[i,day] = score
        
        #Remove these players
        df_now = df_now[~df_now.Name.isin(opt_team)]
        
    #Now find the best team you could possibly make from this player pool
    df_now = df[df.Name.isin(opt_teams)]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.FPTS.values
    w = df_now.Win.values
    o = df_now.Odds.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap]

    #Minimize Winning Odds
    obj = cp.Maximize(x*p)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[x.value>.5].Salary.sum()
    #opt_teams = opt_teams + opt_team
    score = df_now[x.value>.5]['FPTS'].sum()
    print('  Best Team ({:.1f} pts, sal={}): {}'.format(score,sal,opt_team))
    opt_scores.append(score)
    
    #Update df_outlaw_optimals
    df_outlaw_optimals = pd.concat([df_outlaw_optimals,
                                  pd.DataFrame(data={'Optimal Team':[", ".join(opt_team)],
                                                    'Salary':sal,
                                                    'FPTS':score})])
    
        
df_scores['Total'] = df_scores.mean(axis=1)
print('\nOverall Average Score: {:.1f}'.format(df_scores.Total.mean()))
print('Average Optimal Score: {:.1f}'.format(np.mean(opt_scores)))
df_scores
df_outlaw_optimals.index = days
df_outlaw_optimals.index.name = 'Date'
df_outlaw_optimals   

In [ ]:
df_outlaw_optimals.Salary = df_outlaw_optimals.Salary.astype(int)

In [ ]:
df_outlaw_optimals.Salary.mean()

In [ ]:
from IPython.display import display, HTML

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
opt_sal_text= '<h4>Average Optimal Outlaw Tour Salary: {:.1f}</h4>'.format(int(df_outlaw_optimals.Salary.mean()))
full_html = opt_sal_text + df_outlaw_optimals.to_html()
display(HTML(full_html))

In [ ]:
from bokeh.io import show, output_notebook, save, export_png
from bokeh.resources import INLINE
from bokeh.layouts import row, widgetbox, column
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import Button, DataTable, TableColumn, Div
from bokeh.plotting import figure
output_notebook(INLINE)

In [ ]:
p = Div()
p.text = full_html
show(p)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
d = {'x{}'.format(i): range(30) for i in range(10)}

table = pd.DataFrame(d)

plt.figure()

# table
plt.subplot(121)

cell_text = []
for row in range(len(table)):
    cell_text.append(table.iloc[row])

plt.table(cellText=cell_text, colLabels=table.columns, loc='center')
plt.axis('off')

# plot
plt.subplot(222)
plt.plot(table['x1'],table['x2'])

# plot
plt.subplot(224)
plt.plot(table['x1'],table['x2'])

plt.show()

In [ ]:
plt.figure()
table = df_outlaw_optimals.reset_index().copy()
# table
plt.subplot(121)

cell_text = []
for row in range(len(table)):
    cell_text.append(table.iloc[row])

plt.table(cellText=cell_text, colLabels=table.columns, loc='center',
         fontsize=19)
plt.axis('off')

# plot
# plt.subplot(222)
# plt.plot(table['x1'],table['x2'])

# # plot
# plt.subplot(224)
# plt.plot(table['x1'],table['x2'])

plt.show()

In [ ]:
#Max win probability
#Explore truly optimal teams
num_teams_deep = 3
df_scores = pd.DataFrame(data={day:np.zeros(num_teams_deep) for day in days})
opt_scores = []
for day, df in zip(days,df_list):
    print(day)
    df_now = df.copy()
    min_sal = 6200
    max_sal = 12000
    df_now = df_now[(df_now.Salary>=min_sal)&(df_now.Salary<=max_sal)]
    opt_teams = []
    for i in range(num_teams_deep):
        n = len(df_now)
        x = cp.Variable(n,boolean=True)
        s = df_now.Salary.values
        w = df_now.Win.values
        o = df_now.Odds.values
        p = df_now.FPTS.values

        cap = 50000
        constraints = [cp.sum(x) == 6,
                     x*s <= cap]

        #Objective
        #obj = cp.Minimize(x*o)
        obj = cp.Maximize(x*w)

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve()  
        opt_team = df_now['Name'][x.value>.5].tolist()
        sal=df_now['Salary'][x.value>.5].sum()
        opt_teams = opt_teams + opt_team
        score = df_now[x.value>.5]['FPTS'].sum()
        print('  Team {} ({:.1f} pts): {}'.format(i+1,score,opt_team))
        df_scores.at[i,day] = score
        
        #Remove these players
        df_now = df_now[~df_now.Name.isin(opt_team)]
        
    #Now find the best team you could possibly make from this player pool
    df_now = df[df.Name.isin(opt_teams)]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.FPTS.values
    w = df_now.Win.values
    o = df_now.Odds.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap]

    #Minimize Winning Odds
    obj = cp.Maximize(x*p)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[x.value>.5].Salary.sum()
    #opt_teams = opt_teams + opt_team
    score = df_now[x.value>.5]['FPTS'].sum()
    print('  Best Team ({:.1f} pts, sal={}): {}'.format(score,sal,opt_team))
    opt_scores.append(score)
    
        
df_scores['Total'] = df_scores.mean(axis=1)
print('\nOverall Average Score: {:.1f}'.format(df_scores.Total.mean()))
print('Average Optimal Score: {:.1f}'.format(np.mean(opt_scores)))
df_scores

In [ ]:
#Minimize Odds
num_teams_deep = 3
df_scores = pd.DataFrame(data={day:np.zeros(num_teams_deep) for day in days})
opt_scores = []
for day, df in zip(days,df_list):
    print(day)
    df_now = df.copy()
    to_remove = ['JJ Spaun']
    df_now = df_now[~df_now.Name.isin(to_remove)]
    #df_now = df_now[df_now.Name!='Yuki Moriyama']
    min_sal = 6700
    df_now = df_now[df_now.Salary>=min_sal]
    opt_teams = []
    for i in range(num_teams_deep):
        n = len(df_now)
        x = cp.Variable(n,boolean=True)
        s = df_now.Salary.values
        w = df_now.Win.values
        o = df_now.Odds.values

        cap = 50000
        constraints = [cp.sum(x) == 6,
                     x*s <= cap]

        #Minimize Winning Odds
        obj = cp.Minimize(x*o)

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve()  
        opt_team = df_now['Name'][x.value>.5].tolist()
        opt_teams = opt_teams + opt_team
        score = df_now[x.value>.5]['FPTS'].sum()
        print('  Team {} ({:.1f} pts): {}'.format(i+1,score,opt_team))
        df_scores.at[i,day] = score
        
        #Remove these players
        df_now = df_now[~df_now.Name.isin(opt_team)]
        
    #Now find the best team you could possibly make from this player pool
    df_now = df[df.Name.isin(opt_teams)]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.FPTS.values
    w = df_now.Win.values
    o = df_now.Odds.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap]

    #Minimize Winning Odds
    obj = cp.Maximize(x*p)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[x.value>.5].Salary.sum()
    #opt_teams = opt_teams + opt_team
    score = df_now[x.value>.5]['FPTS'].sum()
    print('  Best Team ({:.1f} pts, sal={}): {}'.format(score,sal,opt_team))
    opt_scores.append(score)
    
        
df_scores['Total'] = df_scores.mean(axis=1)
print('\nOverall Average Score: {:.1f}'.format(df_scores.Total.mean()))
print('Average Optimal Score: {:.1f}'.format(np.mean(opt_scores)))
df_scores

In [ ]:
#Check out some kind of value metric to pick top 15 players
df = df_list[2]
df['Value'] = np.power(df.Win,.5) / df.Salary*1e6
df.sort_values(by='Value',ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head(25)

# plot DK pts gained vs odds

In [ ]:
df_gained = pd.DataFrame()
for df in df_list:
    df['gained']=df.FPTS-df.FPTS.mean()
    df_gained=pd.concat([df_gained,
                        df[['Odds','gained']]])
df_gained.iloc[np.r_[0:4,-4:0]]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
df_gained.plot(kind='scatter',
              x='Odds',
              y='gained')

In [ ]:
from scipy.stats import linregress

In [ ]:
L =linregress(x=df_gained)
L

In [ ]:
L.intercept

In [ ]:

from bokeh.io import curdoc, show, output_notebook, save
from bokeh.resources import INLINE
from bokeh.layouts import row, widgetbox, column
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import Button, DataTable, TableColumn
from bokeh.plotting import figure
output_notebook(INLINE)

In [ ]:
from sklearn.linear_model import LinearRegression 
lin = LinearRegression()
from sklearn.preprocessing import PolynomialFeatures  

poly = PolynomialFeatures(degree = 4) 
X=df_gained[['Odds']]
X_poly = poly.fit_transform(X) 

y=df_gained.gained
poly.fit(X_poly, y) 

lin2 = LinearRegression() 
lin2.fit(X_poly, y)

In [ ]:
p = figure()
p.circle(x=df_gained.Odds,
        y=df_gained.gained)
x_plot=np.arange(0,df_gained.Odds.max())
x_plot_poly = poly.fit_transform(pd.DataFrame(x_plot))
y_predicted = lin2.predict(x_plot_poly)
p.line(x=x_plot,y=y_predicted)
p.line(x=x_plot,y=L.intercept + L.slope*x_plot,color='red')
show(p)

In [ ]:
#see what dispersion looks like around expected pts
df_gained['proj']=lin2.predict(poly.fit_transform(df_gained[['Odds']]))
df_gained['diff']=df_gained.gained-df_gained.proj
df_gained.head()

In [ ]:
df_gained.hist('diff',bins=12)

In [ ]:
#what is sd of doff
df_gained['diff'].describe()

In [ ]:
#save the sd for scores
sd = df_gained['diff'].describe().loc['std']
sd

In [ ]:
#Does sd differ for different odds levels?
num_tiers = 10
df_gained['tier'] = pd.qcut(df_gained.Odds,q=num_tiers,labels=[str(i+1) for i in range(num_tiers)])
df_gained.head()

In [ ]:
df_gained.groupby('tier')['diff'].agg(lambda x: np.std(x))

In [ ]:
#Add projected points (Proj) to each df
for i, df in enumerate(df_list):
    df_now = df.copy()
    df_now['Proj'] = lin2.predict(poly.fit_transform(df_now[['Odds']]))
    df_list[i] = df_now
df_list[-1].head()

In [ ]:
#Max projected points
num_teams_deep = 3
df_scores = pd.DataFrame(data={day:np.zeros(num_teams_deep) for day in days})
opt_scores = []
for day, df in zip(days,df_list):
    print(day)
    df_now = df.copy()
    to_remove = ['JJ Spaun']
    df_now = df_now[~df_now.Name.isin(to_remove)]
    #df_now = df_now[df_now.Name!='Yuki Moriyama']
    min_sal = 6400
    df_now = df_now[df_now.Salary>=min_sal]
    
    #Add proj
    #df_now['Proj'] = lin2.predict(poly.fit_transform(df_now[['Odds']]))
    opt_teams = []
    for i in range(num_teams_deep):
        n = len(df_now)
        x = cp.Variable(n,boolean=True)
        s = df_now.Salary.values
        w = df_now.Win.values
        o = df_now.Odds.values
        p = df_now.Proj.values

        cap = 50000
        constraints = [cp.sum(x) == 6,
                     x*s <= cap]

        #Minimize Winning Odds
        #obj = cp.Minimize(x*o)
        obj = cp.Maximize(x*p)

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve()  
        opt_team = df_now['Name'][x.value>.5].tolist()
        opt_teams = opt_teams + opt_team
        score = df_now[x.value>.5]['FPTS'].sum()
        print('  Team {} ({:.1f} pts): {}'.format(i+1,score,opt_team))
        df_scores.at[i,day] = score
        
        #Remove these players
        df_now = df_now[~df_now.Name.isin(opt_team)]
        
    #Now find the best team you could possibly make from this player pool
    df_now = df[df.Name.isin(opt_teams)]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.FPTS.values
    w = df_now.Win.values
    o = df_now.Odds.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap]

    #Minimize Winning Odds
    obj = cp.Maximize(x*p)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[x.value>.5].Salary.sum()
    #opt_teams = opt_teams + opt_team
    score = df_now[x.value>.5]['FPTS'].sum()
    print('  Best Team ({:.1f} pts, sal={}): {}'.format(score,sal,opt_team))
    opt_scores.append(score)
    
        
df_scores['Total'] = df_scores.mean(axis=1)
print('\nOverall Average Score: {:.1f}'.format(df_scores.Total.mean()))
print('Average Optimal Score: {:.1f}'.format(np.mean(opt_scores)))
df_scores

### Save (pickle) trained model

In [ ]:
import pickle

# Dump the trained model with Pickle
pkl_filename = 'outlaw_proj_model.pkl'
# Open the file to save as pkl file
outlaw_proj_model_pkl = open(pkl_filename, 'wb')
pickle.dump(lin2, outlaw_proj_model_pkl)
# Close the pickle instances
outlaw_proj_model_pkl.close()

In [ ]:
#Loading the saved model pickle
outlaw_proj_model_pkl = open(pkl_filename, 'rb')
outlaw_proj_model = pickle.load(outlaw_proj_model_pkl)
print ("Loaded Outlaw model :: ", outlaw_proj_model)

In [ ]:
outlaw_proj_model.predict(x_plot_poly)

# Optimize by number of times players appear in optimal lineups

### First, simulate to add num_opt to each df

In [ ]:
df_list[-1].head()

In [ ]:
def sim_optimal_team(df,sd=10.8):
    #df needs column called Points
    # sd is standard dev for added normal points.  Set lower for less shuffling of teams away from pure max expectation
    #sd = 10.8
    df_now = df.copy()
    df_now['sim_pts']=df.Proj+sd*np.random.randn(len(df_now))
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    sim_pts = df_now.sim_pts.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
             x*s <= cap]
    obj = cp.Maximize(x*sim_pts)
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    return opt_team

sim_optimal_team(df_list[-1])

In [ ]:
#Simulate to add num_opt to each df
num_sims = 1000
for i, df in enumerate(df_list):
    df_now = df.copy()
    df_now['num_opt'] = 0
    for n in range(num_sims):
#         if i%(int(num_sims/4))==0:
#             print('i = {}'.format(i))
        opt_team=sim_optimal_team(df_now,sd=sd)
#         opt_sal=df_all[df_all.Name.isin(opt_team)].Salary.sum()
#         total_sal+=opt_sal
        df_now.num_opt=np.where(df_now.Name.isin(opt_team),
                               df_now.num_opt +1,df_now.num_opt)
    
    df_list[i] = df_now
    print("Done with {} of {} df's".format(i+1,len(df_list)))

In [ ]:
df_list[-1]

In [ ]:
#Max by num_opt
num_teams_deep = 3
df_scores = pd.DataFrame(data={day:np.zeros(num_teams_deep) for day in days})
opt_scores = []
for day, df in zip(days,df_list):
    print(day)
    df_now = df.copy()
    to_remove = ['JJ Spaun']
    df_now = df_now[~df_now.Name.isin(to_remove)]
    #df_now = df_now[df_now.Name!='Yuki Moriyama']
    min_sal = 6000
    df_now = df_now[df_now.Salary>=min_sal]
    
    #Add proj
    #df_now['Proj'] = lin2.predict(poly.fit_transform(df_now[['Odds']]))
    opt_teams = []
    for i in range(num_teams_deep):
        n = len(df_now)
        x = cp.Variable(n,boolean=True)
        s = df_now.Salary.values
        w = df_now.Win.values
        o = df_now.Odds.values
        p = df_now.num_opt.values

        cap = 50000
        constraints = [cp.sum(x) == 6,
                     x*s <= cap]

        #Minimize Winning Odds
        #obj = cp.Minimize(x*o)
        obj = cp.Maximize(x*p)

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve()  
        opt_team = df_now['Name'][x.value>.5].tolist()
        opt_teams = opt_teams + opt_team
        score = df_now[x.value>.5]['FPTS'].sum()
        print('  Team {} ({:.1f} pts): {}'.format(i+1,score,opt_team))
        df_scores.at[i,day] = score
        
        #Remove these players
        df_now = df_now[~df_now.Name.isin(opt_team)]
        
    #Now find the best team you could possibly make from this player pool
    df_now = df[df.Name.isin(opt_teams)]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.FPTS.values
    w = df_now.Win.values
    o = df_now.Odds.values

    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap]

    #Minimize Winning Odds
    obj = cp.Maximize(x*p)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  
    opt_team = df_now['Name'][x.value>.5].tolist()
    sal = df_now[x.value>.5].Salary.sum()
    #opt_teams = opt_teams + opt_team
    score = df_now[x.value>.5]['FPTS'].sum()
    print('  Best Team ({:.1f} pts, sal={}): {}'.format(score,sal,opt_team))
    opt_scores.append(score)
    
        
df_scores['Total'] = df_scores.mean(axis=1)
print('\nOverall Average Score: {:.1f}'.format(df_scores.Total.mean()))
print('Average Optimal Score: {:.1f}'.format(np.mean(opt_scores)))
df_scores